In [1]:
# Import
from sklearn.model_selection import train_test_split
import duckdb, torch, time, os, gc
import numpy as np
import pandas as pd
from pathlib import Path
from scipy.sparse import load_npz, hstack, save_npz

from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import f1_score, accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import ParameterGrid
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

In [2]:
DB_PATH = "D:/db/meta.duckdb"
con = duckdb.connect(DB_PATH)
try:
    con.execute("PRAGMA threads=8;")
except duckdb.InvalidInputException:
    pass

print("Set up ready")

Set up ready


# METADATA PREPARATION

In [ ]:
metadata_tr = con.sql("""SELECT * FROM md1718 WHERE split = 'train'""").df()
metadata_val = con.sql("""SELECT * FROM md1718 WHERE split = 'validation'""").df()
metadata_te = con.sql("""SELECT * FROM md1718 WHERE split = 'test'""").df()

col_to_exclude = ['filename', 'username', 'like_count', 'comment_count', 'caption', 'followers', 'engagement_rate', 'er_log', 'caption_language', 'caption_tfidf', 'caption_bert_clip',
                  'er_bins', 'split', 'in_train_balanced', 'time_utc', 'date_day', 'caption_lang', 'caption_clean']

feature_columns = [col for col in metadata_tr.columns if col not in col_to_exclude]

In [ ]:
y_tr = metadata_tr[['post_id', 'er_bins']]
y_val = metadata_val[['post_id', 'er_bins']]
y_te = metadata_te[['post_id', 'er_bins']]

meta_tr = metadata_tr[feature_columns]
meta_val = metadata_val[feature_columns] 
meta_te = metadata_te[feature_columns] 

# One hot encoding for orientation and category
cat_cols = ['orientation', 'category']
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
meta_train_encoded = encoder.fit_transform(meta_tr[cat_cols])
meta_val_encoded = encoder.transform(meta_val[cat_cols])
meta_test_encoded = encoder.transform(meta_te[cat_cols])

encoded_cols = encoder.get_feature_names_out(cat_cols)
meta_train_encoded = pd.DataFrame(meta_train_encoded, columns=encoded_cols, index=meta_tr.index)
meta_val_encoded = pd.DataFrame(meta_val_encoded, columns=encoded_cols, index=meta_val.index)
meta_test_encoded = pd.DataFrame(meta_test_encoded, columns=encoded_cols, index=meta_te.index)

meta_tr['caption_len_char'] = meta_tr['caption_len_char'].fillna(0)
meta_val['caption_len_char'] = meta_val['caption_len_char'].fillna(0)
meta_te['caption_len_char'] = meta_te['caption_len_char'].fillna(0)

# Scaling numerical variables
scaler = StandardScaler()
num_cols = [
    'width', 'height', 'aspect_ratio', 'area',
    'dow', 'hour_utc', 'month', 'year', 'caption_len_char',
    'n_hashtags', 'n_mentions', 'n_urls', 'n_emojis',
    'followees', 'posts'
]

bin_cols = ['has_caption'] 

meta_train_scaled = pd.DataFrame(
    scaler.fit_transform(meta_tr[num_cols]),
    columns=num_cols,
    index=meta_tr.index
)
meta_val_scaled = pd.DataFrame(
    scaler.transform(meta_val[num_cols]),
    columns=num_cols,
    index=meta_val.index
)
meta_test_scaled = pd.DataFrame(
    scaler.transform(meta_te[num_cols]),
    columns=num_cols,
    index=meta_te.index
)

meta_train_bin = meta_tr[bin_cols].astype(int)
meta_val_bin = meta_val[bin_cols].astype(int)
meta_test_bin = meta_te[bin_cols].astype(int)

# Merge all: scaled numeric + boolean + one-hot
meta_train_final = pd.concat([meta_train_scaled, meta_train_bin, meta_train_encoded], axis=1)
meta_val_final = pd.concat([meta_val_scaled, meta_val_bin, meta_val_encoded], axis=1)
meta_test_final = pd.concat([meta_test_scaled, meta_test_bin, meta_test_encoded], axis=1)
# Add post_id
meta_train_final.insert(0, 'post_id', metadata_tr['post_id'].values)
meta_val_final.insert(0, 'post_id', metadata_val['post_id'].values)

In [7]:
meta_test_final.insert(0, 'post_id', metadata_te['post_id'].values)

In [8]:
meta_test_final.head()

,post_id,width,height,aspect_ratio,area,dow,hour_utc,month,year,caption_len_char,...,orientation_square,category_beauty,category_family,category_fashion,category_fitness,category_food,category_interior,category_other,category_pet,category_travel
0,la.brandon-1936393348858127807,0.281866,1.081142,-0.912237,0.986369,-1.006889,0.241193,1.780745,0.665162,-0.303090,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,la.brandon-1937213763687991478,0.281866,1.090130,-0.912237,0.993974,-0.494989,0.649684,1.780745,0.665162,-0.039784,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,la.brandon-1937218941271111413,0.281866,-0.123249,0.095960,-0.032801,-0.494989,0.649684,1.780745,0.665162,-0.665522,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,la.brandon-1938181925006920700,0.281866,-0.123249,0.095960,-0.032801,0.528810,-1.528935,1.780745,0.665162,0.362919,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,la.brandon-1938620907515771752,0.281866,1.090130,-0.912237,0.993974,0.528810,0.513520,1.780745,0.665162,0.706765,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
print(meta_train_final.shape, meta_val_final.shape, meta_test_final.shape)

(773497, 29) (412325, 29) (423604, 29)


In [10]:
# Free memory
del metadata_tr, metadata_val
del meta_tr, meta_val
del meta_train_scaled, meta_val_scaled
del meta_train_encoded, meta_val_encoded

gc.collect()

2094

In [11]:
meta_train_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 773497 entries, 0 to 773496
Data columns (total 29 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   post_id                773497 non-null  object 
 1   width                  773497 non-null  float64
 2   height                 773497 non-null  float64
 3   aspect_ratio           773497 non-null  float64
 4   area                   773497 non-null  float64
 5   dow                    773497 non-null  float64
 6   hour_utc               773497 non-null  float64
 7   month                  773497 non-null  float64
 8   year                   773497 non-null  float64
 9   caption_len_char       773497 non-null  float64
 10  n_hashtags             773497 non-null  float64
 11  n_mentions             773497 non-null  float64
 12  n_urls                 773497 non-null  float64
 13  n_emojis               773497 non-null  float64
 14  followees              773497 non-nu

In [12]:
meta_train_final.to_csv("D:/dataset/meta_classification/meta_train_final.csv", index=False)
meta_val_final.to_csv("D:/dataset/meta_classification/meta_val_final.csv", index=False)
meta_test_final.to_csv("D:/dataset/meta_classification/meta_test_final.csv", index=False)

In [13]:
y_tr.to_csv("D:/dataset/meta_classification/y_tr.csv", index=False)
y_val.to_csv("D:/dataset/meta_classification/y_va.csv", index=False)
y_te.to_csv("D:/dataset/meta_classification/y_te.csv", index=False)

In [5]:
DB_PATH = "D:/db/meta.duckdb"
con = duckdb.connect(DB_PATH)
try:
    con.execute("PRAGMA threads=8;")
except duckdb.InvalidInputException:
    pass

print("Set up ready")

Set up ready


# TRAIN

In [3]:
# METADATA
meta_train_final = pd.read_csv("D:/dataset/meta_classification/meta_train_final.csv")

# SBERT
train = np.load("D:/dataset/sbert_emb/paraphrase-MiniLM-L6-v2_train_ids_y.npz", allow_pickle=True)
SB_tr = train["embeddings"]
SB_ids_tr = train["ids"]

# EFFICIENT NET
EN_tr_data = np.load("D:/dataset/efficientnetb0_emb/train_data.npz", allow_pickle=True)
EN_tr = EN_tr_data["X"]
EN_tr_ids = EN_tr_data["ids"]

In [4]:
# ALIGN THE THREE SETS
meta_tr = meta_train_final.sort_values("post_id").reset_index(drop=True)
tr_ids_ord = meta_tr["post_id"].values

SB_ids_tr = SB_ids_tr.astype(str)
EN_tr_ids = EN_tr_ids.astype(str)
tr_ids_ord = tr_ids_ord.astype(str)

metadata_tr = con.sql("""
    SELECT post_id, er_bins FROM md1718
    WHERE split = 'train'
""").df().set_index('post_id')

y_tr = metadata_tr.loc[tr_ids_ord, 'er_bins'].values
assert len(y_tr) == len(tr_ids_ord)

# SBERT -> DataFrame indicizzato
df_sbert = pd.DataFrame(SB_tr, index=SB_ids_tr)
df_sbert.index.name = "post_id"

# EfficientNet -> DataFrame indicizzato
df_eff = pd.DataFrame(EN_tr, index=EN_tr_ids)
df_eff.index.name = "post_id"

df_sbert.index = df_sbert.index.astype(str)
df_eff.index   = df_eff.index.astype(str)
metadata_tr.index = metadata_tr.index.astype(str)

# Allineamento perfetto tramite reindex
SBERT_tr_aligned = df_sbert.reindex(tr_ids_ord).values
EN_tr_aligned = df_eff.reindex(tr_ids_ord).values

# Metadata numerici
meta_tr_np = meta_tr.to_numpy()

In [5]:
for idx in np.random.choice(len(tr_ids_ord), 10, replace=False):
    pid = tr_ids_ord[idx]  # the ID used for alignment
    
    # Extract SBERT original label for that post_id
    original_label_sbert = train["y"][np.where(SB_ids_tr == pid)[0][0]]
    
    # Our final label
    aligned_label = y_tr[idx]

    print(f"ID {pid} -> SBERT label: {original_label_sbert}, aligned label: {aligned_label}")

ID jaxedup_-1661166768121876485 -> SBERT label: very_high, aligned label: very_high
ID kathypatalsky-1828618111151180132 -> SBERT label: low, aligned label: low
ID jennydinhofficial-1726460627143991097 -> SBERT label: low, aligned label: low
ID empressdshenna-1752915371617655852 -> SBERT label: very_high, aligned label: very_high
ID thebeauparlour-1778381395752840551 -> SBERT label: high, aligned label: high
ID tiny_house_ideas-1879917602862398442 -> SBERT label: medium, aligned label: medium
ID pencilandpaperco-1887071511689775340 -> SBERT label: very_low, aligned label: very_low
ID savy.jean-1582021008067721506 -> SBERT label: high, aligned label: high
ID joshmair96-1592269944325578069 -> SBERT label: very_low, aligned label: very_low
ID pia.falbo-1757290049832401947 -> SBERT label: high, aligned label: high


In [6]:
for idx in np.random.choice(len(tr_ids_ord), 10, replace=False):
    pid = tr_ids_ord[idx]
    
    # EN original label for that post_id
    original_label_en = EN_tr_data["y"][np.where(EN_tr_ids == pid)[0][0]]
    
    aligned_label = y_tr[idx]

    print(f"ID {pid} -> EN label: {original_label_en}, aligned label: {aligned_label}")

ID the_hungrybeast-1599673526892457200 -> EN label: medium, aligned label: medium
ID alexajeanfitness-1561113583289511037 -> EN label: low, aligned label: low
ID realtgallagher-1780030529388508217 -> EN label: very_high, aligned label: very_high
ID xoxokrispin-1792488909834724604 -> EN label: medium, aligned label: medium
ID dearestmercy-1583443501966678421 -> EN label: low, aligned label: low
ID canary_jane-1769294641598957093 -> EN label: medium, aligned label: medium
ID aaron_wylie10-1746730557322241542 -> EN label: very_high, aligned label: very_high
ID camillelolafit-1691265434270409474 -> EN label: high, aligned label: high
ID mazroudecouvre-1514612765611665859 -> EN label: very_low, aligned label: very_low
ID colourmehappydecorating-1839884047491065851 -> EN label: very_low, aligned label: very_low


In [7]:
def check_alignment(idx):
    pid = tr_ids_ord[idx]
    sbert_ok = np.allclose(SBERT_tr_aligned[idx], df_sbert.loc[pid].values)
    en_ok = np.allclose(EN_tr_aligned[idx], df_eff.loc[pid].values)
    label_ok = (y_tr[idx] == metadata_tr.loc[pid, 'er_bins'])
    return sbert_ok, en_ok, label_ok

results = np.array([check_alignment(i) for i in range(200)])  # check first 200 rows

print("SBERT alignment OK:", results[:,0].all())
print("EN alignment OK:", results[:,1].all())
print("Label alignment OK:", results[:,2].all())


SBERT alignment OK: True
EN alignment OK: True
Label alignment OK: True


In [8]:
# Checks
assert SBERT_tr_aligned.shape[0] == len(meta_tr)
assert EN_tr_aligned.shape[0] == len(meta_tr)

print(tr_ids_ord[:5])
print(df_sbert.reindex(tr_ids_ord).index[:5])
print(df_eff.reindex(tr_ids_ord).index[:5])
print(len(tr_ids_ord), SBERT_tr_aligned.shape[0], EN_tr_aligned.shape[0], meta_tr_np.shape[0], len(y_tr))

['100pintas-1769662389073991144' '100pintas-1782702664733979876'
 '100pintas-1797067212467389817' '100pintas-1807955339238986900'
 '100pintas-1808039696742034708']
Index(['100pintas-1769662389073991144', '100pintas-1782702664733979876',
       '100pintas-1797067212467389817', '100pintas-1807955339238986900',
       '100pintas-1808039696742034708'],
      dtype='object', name='post_id')
Index(['100pintas-1769662389073991144', '100pintas-1782702664733979876',
       '100pintas-1797067212467389817', '100pintas-1807955339238986900',
       '100pintas-1808039696742034708'],
      dtype='object', name='post_id')
773497 773497 773497 773497 773497


In [9]:
assert (meta_tr["post_id"].values == tr_ids_ord).all()

# Per SBERT
assert (df_sbert.loc[tr_ids_ord].index.values == tr_ids_ord).all()

# Per EfficientNet
assert (df_eff.loc[tr_ids_ord].index.values == tr_ids_ord).all()

print("SBERT missing:", df_sbert.reindex(tr_ids_ord).isna().any(axis=1).sum())
print("EN missing:", df_eff.reindex(tr_ids_ord).isna().any(axis=1).sum())

SBERT missing: 0
EN missing: 0


In [10]:
post_id_tr = meta_train_final['post_id'].values
meta_tr_num = meta_train_final.drop(columns=['post_id']).values

meta_tr_num = meta_tr_num.astype('float32')
SBERT_tr_aligned = SBERT_tr_aligned.astype('float32')
EN_tr_aligned = EN_tr_aligned.astype('float32')

X_tr = np.concatenate((meta_tr_num, SBERT_tr_aligned, EN_tr_aligned), axis = 1)

In [11]:
print(X_tr.shape, meta_tr_num.shape, SBERT_tr_aligned.shape, EN_tr_aligned.shape)

(773497, 1692) (773497, 28) (773497, 384) (773497, 1280)


In [12]:
np.save("D:/dataset/multimodal2/X_train.npy", X_tr)
np.save("D:/dataset/multimodal2/y_tr_5.npy", y_tr)
np.save("D:/dataset/multimodal2/post_id_tr.npy", post_id_tr)

# VALIDATION

In [3]:
# METADATA
meta_val_final = pd.read_csv("D:/dataset/meta_classification/meta_val_final.csv")

# SBERT
val = np.load("D:/dataset/sbert_emb/paraphrase-MiniLM-L6-v2_val_ids_y.npz", allow_pickle=True)
SB_va = val["embeddings"]
SB_ids_va = val["ids"]

# EFFICIENT NET
EN_va_data = np.load("D:/dataset/efficientnetb0_emb/val_data.npz", allow_pickle=True)
EN_va = EN_va_data["X"]
EN_va_ids = EN_va_data["ids"]

# ALIGN THE THREE SETS
meta_va = meta_val_final.sort_values("post_id").reset_index(drop=True)
val_ids_ord = meta_va["post_id"].values

SB_ids_va = SB_ids_va.astype(str)
EN_va_ids = EN_va_ids.astype(str)
val_ids_ord = val_ids_ord.astype(str)

In [4]:
metadata_va = con.sql("""
    SELECT post_id, er_bins FROM md1718
    WHERE split = 'validation'
""").df().set_index('post_id')

# allineamento diretto ai post_id in X
y_va = metadata_va.loc[val_ids_ord, 'er_bins'].values
assert len(y_va) == len(val_ids_ord)

# SBERT -> DataFrame indicizzato
df_sbert = pd.DataFrame(SB_va, index=SB_ids_va)
df_sbert.index.name = "post_id"

# EfficientNet -> DataFrame indicizzato
df_eff = pd.DataFrame(EN_va, index=EN_va_ids)
df_eff.index.name = "post_id"

df_sbert.index = df_sbert.index.astype(str)
df_eff.index   = df_eff.index.astype(str)
metadata_va.index = metadata_va.index.astype(str)

# Allineamento perfetto tramite reindex
SBERT_va_aligned = df_sbert.reindex(val_ids_ord).values
EN_va_aligned = df_eff.reindex(val_ids_ord).values

# Metadata numerici
meta_va_np = meta_va.to_numpy()

In [5]:
# Checks
assert SBERT_va_aligned.shape[0] == len(meta_va)
assert EN_va_aligned.shape[0] == len(meta_va)

print(val_ids_ord[:5])
print(df_sbert.reindex(val_ids_ord).index[:5])
print(df_eff.reindex(val_ids_ord).index[:5])
print(len(val_ids_ord), SBERT_va_aligned.shape[0], EN_va_aligned.shape[0], meta_va_np.shape[0], len(y_va))

['00_rocketgirl-1905372469183881641' '00_rocketgirl-1905378100137554269'
 '00_rocketgirl-1906985289541418664' '00_rocketgirl-1912887291135668511'
 '00_rocketgirl-1913179760499084122']
Index(['00_rocketgirl-1905372469183881641',
       '00_rocketgirl-1905378100137554269',
       '00_rocketgirl-1906985289541418664',
       '00_rocketgirl-1912887291135668511',
       '00_rocketgirl-1913179760499084122'],
      dtype='object', name='post_id')
Index(['00_rocketgirl-1905372469183881641',
       '00_rocketgirl-1905378100137554269',
       '00_rocketgirl-1906985289541418664',
       '00_rocketgirl-1912887291135668511',
       '00_rocketgirl-1913179760499084122'],
      dtype='object', name='post_id')
412325 412325 412325 412325 412325


In [6]:
assert (meta_va["post_id"].values == val_ids_ord).all()

# Per SBERT
assert (df_sbert.loc[val_ids_ord].index.values == val_ids_ord).all()

# Per EfficientNet
assert (df_eff.loc[val_ids_ord].index.values == val_ids_ord).all()

print("SBERT missing:", df_sbert.reindex(val_ids_ord).isna().any(axis=1).sum())
print("EN missing:", df_eff.reindex(val_ids_ord).isna().any(axis=1).sum())

SBERT missing: 0
EN missing: 0


In [7]:
post_id_va = meta_val_final['post_id'].values
meta_va_num = meta_val_final.drop(columns=['post_id']).values

meta_va_num = meta_va_num.astype('float32')
SBERT_va_aligned = SBERT_va_aligned.astype('float32')
EN_va_aligned = EN_va_aligned.astype('float32')

X_va = np.concatenate((meta_va_num, SBERT_va_aligned, EN_va_aligned), axis = 1)

In [8]:
print(X_va.shape, meta_va_num.shape, SBERT_va_aligned.shape, EN_va_aligned.shape)

(412325, 1692) (412325, 28) (412325, 384) (412325, 1280)


In [9]:
np.save("D:/dataset/multimodal2/X_val.npy", X_va)
np.save("D:/dataset/multimodal2/y_va_5.npy", y_va)
np.save("D:/dataset/multimodal2/post_id_va.npy", post_id_va)

# TEST

In [3]:
# METADATA
meta_test_final = pd.read_csv("D:/dataset/meta_classification/meta_test_final.csv")

# SBERT
test = np.load("D:/dataset/sbert_emb/paraphrase-MiniLM-L6-v2_test_ids_y.npz", allow_pickle=True)
SB_te = test["embeddings"]
SB_ids_te = test["ids"]

# EFFICIENT NET

EN_te_data = np.load("D:/dataset/efficientnetb0_emb/test_data.npz", allow_pickle=True)
EN_te = EN_te_data["X"]
EN_te_ids = EN_te_data["ids"]

# ALIGN THE THREE SETS
meta_te = meta_test_final.sort_values("post_id").reset_index(drop=True)
te_ids_ord = meta_te["post_id"].values


SB_ids_te = SB_ids_te.astype(str)
EN_te_ids = EN_te_ids.astype(str)
te_ids_ord = te_ids_ord.astype(str)

# Recupero le label y
metadata_te = con.sql("""
    SELECT post_id, er_bins FROM md1718
    WHERE split = 'test'
""").df().set_index('post_id')

y_te = metadata_te.loc[te_ids_ord, 'er_bins'].values
assert len(y_te) == len(te_ids_ord)


# SBERT -> DataFrame indicizzato
df_sbert = pd.DataFrame(SB_te, index=SB_ids_te)
df_sbert.index.name = "post_id"

# EfficientNet -> DataFrame indicizzato
df_eff = pd.DataFrame(EN_te, index=EN_te_ids)
df_eff.index.name = "post_id"

df_sbert.index = df_sbert.index.astype(str)
df_eff.index   = df_eff.index.astype(str)
metadata_te.index = metadata_te.index.astype(str)

# Allineamento perfetto tramite reindex
SBERT_te_aligned = df_sbert.reindex(te_ids_ord).values
EN_te_aligned = df_eff.reindex(te_ids_ord).values

# Metadata numerici
meta_te_np = meta_te.to_numpy()


In [4]:
# Checks
assert SBERT_te_aligned.shape[0] == len(meta_te)
assert EN_te_aligned.shape[0] == len(meta_te)

print(te_ids_ord[:5])
print(df_sbert.reindex(te_ids_ord).index[:5])
print(df_eff.reindex(te_ids_ord).index[:5])
print(len(te_ids_ord), SBERT_te_aligned.shape[0], EN_te_aligned.shape[0], meta_te_np.shape[0], len(y_te))

['00_rocketgirl-1924887425826593106' '00_rocketgirl-1926960433592504542'
 '00_rocketgirl-1931680985582309140' '00_rocketgirl-1936081881737003967'
 '00_rocketgirl-1942428720468033285']
Index(['00_rocketgirl-1924887425826593106',
       '00_rocketgirl-1926960433592504542',
       '00_rocketgirl-1931680985582309140',
       '00_rocketgirl-1936081881737003967',
       '00_rocketgirl-1942428720468033285'],
      dtype='object', name='post_id')
Index(['00_rocketgirl-1924887425826593106',
       '00_rocketgirl-1926960433592504542',
       '00_rocketgirl-1931680985582309140',
       '00_rocketgirl-1936081881737003967',
       '00_rocketgirl-1942428720468033285'],
      dtype='object', name='post_id')
423604 423604 423604 423604 423604


In [5]:
assert (meta_te["post_id"].values == te_ids_ord).all()

# Per SBERT
assert (df_sbert.loc[te_ids_ord].index.values == te_ids_ord).all()

# Per EfficientNet
assert (df_eff.loc[te_ids_ord].index.values == te_ids_ord).all()

print("SBERT missing:", df_sbert.reindex(te_ids_ord).isna().any(axis=1).sum())
print("EN missing:", df_eff.reindex(te_ids_ord).isna().any(axis=1).sum())

SBERT missing: 0
EN missing: 0


In [6]:
post_id_te = meta_test_final['post_id'].values
meta_te_num = meta_test_final.drop(columns=['post_id']).values

meta_te_num = meta_te_num.astype('float32')
SBERT_te_aligned = SBERT_te_aligned.astype('float32')
EN_te_aligned = EN_te_aligned.astype('float32')

In [7]:
X_te = np.concatenate((meta_te_num, SBERT_te_aligned, EN_te_aligned), axis = 1)

In [8]:
print(X_te.shape, meta_te_num.shape, SBERT_te_aligned.shape, EN_te_aligned.shape)

(423604, 1692) (423604, 28) (423604, 384) (423604, 1280)


In [9]:
np.save("D:/dataset/multimodal2/X_test.npy", X_te)
np.save("D:/dataset/multimodal2/y_te_5.npy", y_te)
np.save("D:/dataset/multimodal2/post_id_te.npy", post_id_te)

# LOAD FUSED DATA

In [2]:
X_tr = np.load("D:/dataset/multimodal2/X_train.npy", allow_pickle = True).astype(np.float32)
y_tr = np.load("D:/dataset/multimodal2/y_tr_5.npy", allow_pickle = True)

X_va = np.load("D:/dataset/multimodal2/X_val.npy", allow_pickle = True).astype(np.float32)
y_va = np.load("D:/dataset/multimodal2/y_va_5.npy", allow_pickle = True)

In [3]:
# SGD
param_grid = {
    "alpha": [1e-5, 1e-4, 1e-3],
    "class_weight": [None, "balanced"],
    }

results = []
best_score = -np.inf
best_params = None


for params in ParameterGrid(param_grid):
    print(f"\nCombination: {params}")

    clf = SGDClassifier(
        loss="hinge",            
        penalty="l2",            
        **params,
        average = True,
        random_state=42,
        max_iter=200,
        tol=1e-3,
        early_stopping = True
    )

    clf.fit(X_tr, y_tr)

    y_val_pred = clf.predict(X_va)

    macro_f1 = f1_score(y_va, y_val_pred, average="macro")
    acc = accuracy_score(y_va, y_val_pred)

    print(f"macro-F1 (val): {macro_f1} | accuracy (val): {acc}")

    results.append({
        "alpha": params["alpha"],
        "class_weight": params["class_weight"],
        "val_macro_f1": macro_f1,
        "val_accuracy": acc,
    })

    if macro_f1 > best_score:
        best_score = macro_f1
        best_params = params

print("\nBest hyperparameter configuration:")
print(best_params)
print("Validation macro-F1:", best_score)

results_df = pd.DataFrame(results).sort_values("val_macro_f1", ascending=False)
print("\nOrdered results by macro-F1 (validation):")
print(results_df)


Combination: {'alpha': 1e-05, 'class_weight': None}
macro-F1 (val): 0.2536337075796995 | accuracy (val): 0.2643836779239677

Combination: {'alpha': 1e-05, 'class_weight': 'balanced'}
macro-F1 (val): 0.17629061546710947 | accuracy (val): 0.2447559570726975

Combination: {'alpha': 0.0001, 'class_weight': None}
macro-F1 (val): 0.2374582312180352 | accuracy (val): 0.2547965803674286

Combination: {'alpha': 0.0001, 'class_weight': 'balanced'}
macro-F1 (val): 0.24654819957809387 | accuracy (val): 0.2649827199417935

Combination: {'alpha': 0.001, 'class_weight': None}
macro-F1 (val): 0.25663301285392937 | accuracy (val): 0.2616212938822531

Combination: {'alpha': 0.001, 'class_weight': 'balanced'}
macro-F1 (val): 0.25604943095129123 | accuracy (val): 0.266602801188383

Best hyperparameter configuration:
{'alpha': 0.001, 'class_weight': None}
Validation macro-F1: 0.25663301285392937

Ordered results by macro-F1 (validation):
     alpha class_weight  val_macro_f1  val_accuracy
4  0.00100      

In [3]:
# NAIVE BAYES - GAUSSIAN

param_grid_nb = {
    "var_smoothing": [1e-9, 1e-8, 1e-7, 1e-6]
}

results = []
best_score = -np.inf
best_params = None

for params in ParameterGrid(param_grid_nb):
    print(f"\nCombination: {params}")

    clf = GaussianNB(**params)

    # Fit su TRAIN
    clf.fit(X_tr, y_tr)

    # Valutazione su VALIDATION
    y_val_pred = clf.predict(X_va)

    macro_f1 = f1_score(y_va, y_val_pred, average="macro")
    acc = accuracy_score(y_va, y_val_pred)

    print(f"macro-F1 (val): {macro_f1:.4f} | accuracy (val): {acc:.4f}")

    results.append({
        "var_smoothing": params["var_smoothing"],
        "val_macro_f1": macro_f1,
        "val_accuracy": acc,
    })

    # Aggiorno il best model in base alla macro-F1
    if macro_f1 > best_score:
        best_score = macro_f1
        best_params = params

print("\nBest hyperparameter configuration:")
print(best_params)
print("Validation macro-F1:", best_score)

results_df = pd.DataFrame(results).sort_values("val_macro_f1", ascending=False)
print("\nOrdered results by macro-F1 (validation):")
print(results_df)


Combination: {'var_smoothing': 1e-09}
macro-F1 (val): 0.2132 | accuracy (val): 0.2643

Combination: {'var_smoothing': 1e-08}
macro-F1 (val): 0.2132 | accuracy (val): 0.2643

Combination: {'var_smoothing': 1e-07}
macro-F1 (val): 0.2132 | accuracy (val): 0.2643

Combination: {'var_smoothing': 1e-06}
macro-F1 (val): 0.2132 | accuracy (val): 0.2643

Best hyperparameter configuration:
{'var_smoothing': 1e-09}
Validation macro-F1: 0.21320636225167475

Ordered results by macro-F1 (validation):
   var_smoothing  val_macro_f1  val_accuracy
0   1.000000e-09      0.213206      0.264328
1   1.000000e-08      0.213205      0.264325
2   1.000000e-07      0.213203      0.264323
3   1.000000e-06      0.213188      0.264306


In [5]:
# RANDOM FOREST
param_grid_rf = {
    "n_estimators": [30, 50, 80],
    "max_depth": [8, 10, 12],
    "min_samples_leaf": [2, 5],
    "max_features": [0.05, "sqrt"],
}

results = []
best_score = -np.inf
best_params = None

for params in ParameterGrid(param_grid_rf):
    print(f"\nCombination: {params}")

    clf = RandomForestClassifier(
        **params,
        n_jobs=-1,
        random_state=42
    )

    clf.fit(X_tr, y_tr)

    y_val_pred = clf.predict(X_va)

    macro_f1 = f1_score(y_va, y_val_pred, average="macro")
    acc = accuracy_score(y_va, y_val_pred)

    print(f"macro-F1 (val): {macro_f1:.4f} | accuracy (val): {acc:.4f}")

    results.append({
        "n_estimators": params["n_estimators"],
        "max_depth": params["max_depth"],
        "min_samples_leaf": params["min_samples_leaf"],
        "max_features": params["max_features"],
        "val_macro_f1": macro_f1,
        "val_accuracy": acc,
    })

    if macro_f1 > best_score:
        best_score = macro_f1
        best_params = params

print("\nBest hyperparameter configuration (Random Forest):")
print(best_params)
print("Validation macro-F1:", best_score)

results_df_rf = pd.DataFrame(results).sort_values("val_macro_f1", ascending=False)
print("\nOrdered results by macro-F1 (validation):")
print(results_df_rf)

# Among these the best is 
# Combination: {'max_depth': 10, 'max_features': 0.05, 'min_samples_leaf': 5, 'n_estimators': 30}
# macro-F1 (val): 0.2244 | accuracy (val): 0.2418


Combination: {'max_depth': 8, 'max_features': 0.05, 'min_samples_leaf': 2, 'n_estimators': 30}
macro-F1 (val): 0.2068 | accuracy (val): 0.2385

Combination: {'max_depth': 8, 'max_features': 0.05, 'min_samples_leaf': 2, 'n_estimators': 50}
macro-F1 (val): 0.2046 | accuracy (val): 0.2387

Combination: {'max_depth': 8, 'max_features': 0.05, 'min_samples_leaf': 2, 'n_estimators': 80}
macro-F1 (val): 0.2034 | accuracy (val): 0.2387

Combination: {'max_depth': 8, 'max_features': 0.05, 'min_samples_leaf': 5, 'n_estimators': 30}
macro-F1 (val): 0.2077 | accuracy (val): 0.2384

Combination: {'max_depth': 8, 'max_features': 0.05, 'min_samples_leaf': 5, 'n_estimators': 50}
macro-F1 (val): 0.2052 | accuracy (val): 0.2385

Combination: {'max_depth': 8, 'max_features': 0.05, 'min_samples_leaf': 5, 'n_estimators': 80}
macro-F1 (val): 0.2029 | accuracy (val): 0.2383

Combination: {'max_depth': 8, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'n_estimators': 30}
macro-F1 (val): 0.2009 | accuracy (val

KeyboardInterrupt: 

In [3]:
# RANDOM FOREST
param_grid_rf = {
    "n_estimators": [30, 50, 80],
    "max_depth": [12],
    "min_samples_leaf": [2, 5],
    "max_features": [0.05, "sqrt"],
}

results = []
best_score = -np.inf
best_params = None

for params in ParameterGrid(param_grid_rf):
    print(f"\nCombination: {params}")

    clf = RandomForestClassifier(
        **params,
        n_jobs=-1,
        random_state=42
    )

    clf.fit(X_tr, y_tr)

    y_val_pred = clf.predict(X_va)

    macro_f1 = f1_score(y_va, y_val_pred, average="macro")
    acc = accuracy_score(y_va, y_val_pred)

    print(f"macro-F1 (val): {macro_f1:.4f} | accuracy (val): {acc:.4f}")

    results.append({
        "n_estimators": params["n_estimators"],
        "max_depth": params["max_depth"],
        "min_samples_leaf": params["min_samples_leaf"],
        "max_features": params["max_features"],
        "val_macro_f1": macro_f1,
        "val_accuracy": acc,
    })

    if macro_f1 > best_score:
        best_score = macro_f1
        best_params = params

print("\nBest hyperparameter configuration (Random Forest):")
print(best_params)
print("Validation macro-F1:", best_score)

results_df_rf = pd.DataFrame(results).sort_values("val_macro_f1", ascending=False)
print("\nOrdered results by macro-F1 (validation):")
print(results_df_rf)



Combination: {'max_depth': 12, 'max_features': 0.05, 'min_samples_leaf': 2, 'n_estimators': 30}
macro-F1 (val): 0.2349 | accuracy (val): 0.2422

Combination: {'max_depth': 12, 'max_features': 0.05, 'min_samples_leaf': 2, 'n_estimators': 50}
macro-F1 (val): 0.2325 | accuracy (val): 0.2429

Combination: {'max_depth': 12, 'max_features': 0.05, 'min_samples_leaf': 2, 'n_estimators': 80}
macro-F1 (val): 0.2309 | accuracy (val): 0.2437

Combination: {'max_depth': 12, 'max_features': 0.05, 'min_samples_leaf': 5, 'n_estimators': 30}
macro-F1 (val): 0.2320 | accuracy (val): 0.2407

Combination: {'max_depth': 12, 'max_features': 0.05, 'min_samples_leaf': 5, 'n_estimators': 50}
macro-F1 (val): 0.2318 | accuracy (val): 0.2427

Combination: {'max_depth': 12, 'max_features': 0.05, 'min_samples_leaf': 5, 'n_estimators': 80}
macro-F1 (val): 0.2306 | accuracy (val): 0.2443

Combination: {'max_depth': 12, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'n_estimators': 30}
macro-F1 (val): 0.2281 | accura

In [4]:
# XGBOOST

# Convert the labels into numbers
le = LabelEncoder()
y_tr_enc = le.fit_transform(y_tr)
y_val_enc = le.transform(y_va)


param_grid_xgb = {
    "n_estimators": [100, 150], 
    "max_depth": [4, 6], 
    "learning_rate": [0.1], 
    "subsample": [0.8], 
    "colsample_bytree": [0.5], 
    "gamma": [0, 1], 
    "reg_lambda": [1], 
}

results = []
best_score = -np.inf
best_params = None

for params in ParameterGrid(param_grid_xgb):
    print(f"\nCombination: {params}")

    clf = XGBClassifier(
        **params,
        objective="multi:softmax",
        num_class=len(np.unique(y_tr_enc)),
        tree_method="hist",
        eval_metric="mlogloss",
        n_jobs=-1,
        random_state=42,
        verbosity=0,
    )

    clf.fit(X_tr, y_tr_enc)

    y_val_pred = clf.predict(X_va)

    macro_f1 = f1_score(y_val_enc, y_val_pred, average="macro")
    acc = accuracy_score(y_val_enc, y_val_pred)

    print(f"macro-F1 (val): {macro_f1:.4f} | accuracy (val): {acc:.4f}")

    results.append({
        **params,
        "val_macro_f1": macro_f1,
        "val_accuracy": acc,
    })

    if macro_f1 > best_score:
        best_score = macro_f1
        best_params = params

print("\nBest hyperparameter configuration (XGBoost):")
print(best_params)
print("Validation macro-F1:", best_score)

results_df_xgb = pd.DataFrame(results).sort_values("val_macro_f1", ascending=False)
print("\nOrdered results:")
print(results_df_xgb)


Combination: {'colsample_bytree': 0.5, 'gamma': 0, 'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100, 'reg_lambda': 1, 'subsample': 0.8}
macro-F1 (val): 0.2504 | accuracy (val): 0.2576

Combination: {'colsample_bytree': 0.5, 'gamma': 0, 'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 150, 'reg_lambda': 1, 'subsample': 0.8}
macro-F1 (val): 0.2549 | accuracy (val): 0.2626

Combination: {'colsample_bytree': 0.5, 'gamma': 0, 'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 100, 'reg_lambda': 1, 'subsample': 0.8}
macro-F1 (val): 0.2584 | accuracy (val): 0.2639

Combination: {'colsample_bytree': 0.5, 'gamma': 0, 'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 150, 'reg_lambda': 1, 'subsample': 0.8}
macro-F1 (val): 0.2621 | accuracy (val): 0.2687

Combination: {'colsample_bytree': 0.5, 'gamma': 1, 'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100, 'reg_lambda': 1, 'subsample': 0.8}
macro-F1 (val): 0.2504 | accuracy (val): 0.2576

Combination: {'colsample_bytr

# PERFORMANCE ON TEST SET

In [2]:
X_tr = np.load("D:/dataset/multimodal2/X_train.npy", allow_pickle = True).astype(np.float32)
y_tr = np.load("D:/dataset/multimodal2/y_tr_5.npy", allow_pickle = True)

X_va = np.load("D:/dataset/multimodal2/X_val.npy", allow_pickle = True).astype(np.float32)
y_va = np.load("D:/dataset/multimodal2/y_va_5.npy", allow_pickle = True)

X_trva = np.concatenate((X_tr, X_va), axis = 0)
y_trva = np.concatenate((y_tr, y_va), axis = 0)

np.save("D:/dataset/multimodal2/X_trainval.npy", X_trva)
np.save("D:/dataset/multimodal2/y_trva_5.npy", y_trva)

In [ ]:
X_tr = np.load("D:/dataset/multimodal2/X_trainval.npy", allow_pickle = True).astype(np.float32)
y_tr = np.load("D:/dataset/multimodal2/y_trva_5.npy", allow_pickle = True)

X_te = np.load("D:/dataset/multimodal2/X_test.npy", allow_pickle = True).astype(np.float32)
y_te = np.load("D:/dataset/multimodal2/y_te_5.npy", allow_pickle = True)

In [3]:
cfg = SGDClassifier(
        loss="hinge",
        penalty="l2",
        alpha = 0.001,
        average = True,
        class_weight = None,
        random_state=42,
        max_iter=500,
        tol=1e-3,
    )


cfg.fit(X_tr, y_tr)
y_te_pred = cfg.predict(X_te)
macro_f1 = f1_score(y_te, y_te_pred, average="macro")
acc = accuracy_score(y_te, y_te_pred)

print(f"macro-F1 (test): {macro_f1:.4f} | accuracy (test): {acc:.4f}")

macro-F1 (test): 0.2530 | accuracy (test): 0.2591


In [3]:
# Convert the labels into numbers
le = LabelEncoder()
y_tr_enc = le.fit_transform(y_tr)
y_te_enc = le.transform(y_te)

cfgs = [
    RandomForestClassifier(
        max_depth=12, max_features=0.05, min_samples_leaf=2, n_estimators=30, n_jobs=-1, random_state=42
    ),
    XGBClassifier(colsample_bytree = 0.5, gamma = 0, learning_rate = 0.1, max_depth= 6, n_estimators= 150, reg_lambda= 1, subsample= 0.8,
        objective="multi:softmax",
        num_class=len(np.unique(y_tr_enc)),
        tree_method="hist", eval_metric="mlogloss",
        n_jobs=-1, random_state=42, verbosity=0
    )
]

for cfg in cfgs:
    print(f"\nConfiguration: {cfg}")

    # XGB requires a numerical target
    if isinstance(cfg, XGBClassifier):
        cfg.fit(X_tr, y_tr_enc)
        y_te_pred = cfg.predict(X_te)
        macro_f1 = f1_score(y_te_enc, y_te_pred, average="macro")
        acc = accuracy_score(y_te_enc, y_te_pred)

    else:
        cfg.fit(X_tr, y_tr)
        y_te_pred = cfg.predict(X_te)
        macro_f1 = f1_score(y_te, y_te_pred, average="macro")
        acc = accuracy_score(y_te, y_te_pred)

    print(f"macro-F1 (test): {macro_f1:.4f} | accuracy (test): {acc:.4f}")


Configuration: RandomForestClassifier(max_depth=12, max_features=0.05, min_samples_leaf=2,
                       n_estimators=30, n_jobs=-1, random_state=42)
macro-F1 (test): 0.2312 | accuracy (test): 0.2391

Configuration: XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.5, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, feature_weights=None, gamma=0,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=150, n_jobs=-1, num_class=5, ...)
macro-F1 (test): 0.2621 | acc

In [10]:
# NAIVE BAYES
import scipy.sparse as sp

batch_size = 256
classes = np.unique(y_tr)

clf = GaussianNB(var_smoothing=1e-09)

# FIT minibatch
for start in range(0, X_tr.shape[0], batch_size):
    end = min(start + batch_size, X_tr.shape[0])

    X_slice = X_tr[start:end]
    Xb = X_slice.toarray() if sp.issparse(X_slice) else X_slice

    yb = y_tr[start:end]

    if start == 0:
        clf.partial_fit(Xb, yb, classes=classes)
    else:
        clf.partial_fit(Xb, yb)

    del Xb, yb, X_slice
    gc.collect()

# PREDICT minibatch (SU X_te!)
y_te_pred = []

for start in range(0, X_te.shape[0], batch_size):
    end = min(start + batch_size, X_te.shape[0])

    X_slice = X_te[start:end]
    Xb = X_slice.toarray() if sp.issparse(X_slice) else X_slice

    y_te_pred.append(clf.predict(Xb))

    del Xb, X_slice
    gc.collect()

y_te_pred = np.concatenate(y_te_pred)

macro_f1 = f1_score(y_te, y_te_pred, average="macro")
acc = accuracy_score(y_te, y_te_pred)

print(f"macro-F1 (test): {macro_f1:.4f} | accuracy (test): {acc:.4f}")


macro-F1 (test): 0.2148 | accuracy (test): 0.2655
